In [11]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Cargar datos
data = pd.read_parquet("../data/processed/data_semanal.parquet")

# Filtrar un SKU–talla–sucursal
data_filt = data[(data["cod_producto"] == 623406) & 
              (data["cod_talla"] == 105) & 
              (data["cod_sucursal"] == 17)].sort_values("inicio_semana")

print(f"Registros totales: {len(data_filt)}")
print(f"Distribución de ventas: {data_filt['venta_semana'].describe()}")

# Crear variable binaria: venta o no venta
data_filt["venta_binaria"] = (data_filt["venta_semana"] > 0).astype(int)
print(f"Proporción de semanas con venta: {data_filt['venta_binaria'].mean():.2%}")

# Diagnóstico inicial
print("\n=== DIAGNÓSTICO INICIAL ===")
print(f"Semanas con venta: {data_filt['venta_binaria'].sum()}")
print(f"Semanas sin venta: {(data_filt['venta_binaria'] == 0).sum()}")

# Crear lags de ventas
for i in range(1, 5):
  data_filt[f"lag_{i}"] = data_filt["venta_binaria"].shift(i)

# Variables temporales mejoradas
data_filt["semana"] = data_filt["inicio_semana"].dt.isocalendar().week
data_filt["mes"] = data_filt["inicio_semana"].dt.month
data_filt["semana_sin"] = np.sin(2 * np.pi * data_filt["semana"] / 52)
data_filt["semana_cos"] = np.cos(2 * np.pi * data_filt["semana"] / 52)

# Estadísticas móviles
data_filt["venta_media_4w"] = data_filt["venta_binaria"].rolling(4, min_periods=1).mean()
data_filt["venta_suma_4w"] = data_filt["venta_binaria"].rolling(4, min_periods=1).sum()

# Eliminar nulos generados por lags
data_model = data_filt.dropna()
print(f"Registros para modelar: {len(data_model)}")

# Features expandidas
features = ["lag_1", "lag_2", "lag_3", "lag_4", "semana", "mes", 
         "semana_sin", "semana_cos", "venta_media_4w", "venta_suma_4w"]
target = "venta_binaria"

X = data_model[features]
y = data_model[target]

# Validación cruzada temporal con manejo de casos extremos
tscv = TimeSeriesSplit(n_splits=min(3, len(data_model)//2))
scores = []

print("\n=== VALIDACIÓN CRUZADA ===")
for i, (train_idx, test_idx) in enumerate(tscv.split(X)):
   X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
   y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
   
   print(f"Fold {i+1}: Train={len(X_train)}, Test={len(X_test)}")
   print(f"  Train - Ventas: {y_train.sum()}/{len(y_train)} ({y_train.mean():.1%})")
   print(f"  Test - Ventas: {y_test.sum()}/{len(y_test)} ({y_test.mean():.1%})")
   
   model = LGBMClassifier(
       n_estimators=100,
       learning_rate=0.1,
       max_depth=3,
       random_state=42,
       verbose=-1
   )
   model.fit(X_train, y_train)
   y_pred = model.predict(X_test)
   accuracy = accuracy_score(y_test, y_pred)
   scores.append(accuracy)
   print(f"  Precisión: {accuracy:.1%}")

print(f"\nPrecisión promedio: {np.mean(scores):.2%} (+/- {np.std(scores)*2:.2%})")

# Entrenar modelo final con todos los datos
model = LGBMClassifier(
   n_estimators=100,
   learning_rate=0.1,
   max_depth=3,
   random_state=42,
   verbose=-1
)
model.fit(X, y)

# Predecir próxima semana
last_row = data_model.iloc[-1:]
next_week = last_row["semana"].values[0] + 1
next_month = last_row["mes"].values[0] if next_week <= 52 else ((last_row["mes"].values[0] % 12) + 1)
next_week = next_week if next_week <= 52 else 1

next_input = pd.DataFrame({
   "lag_1": [last_row["venta_binaria"].values[0]],
   "lag_2": [last_row["lag_1"].values[0]],
   "lag_3": [last_row["lag_2"].values[0]],
   "lag_4": [last_row["lag_3"].values[0]],
   "semana": [next_week],
   "mes": [next_month],
   "semana_sin": [np.sin(2 * np.pi * next_week / 52)],
   "semana_cos": [np.cos(2 * np.pi * next_week / 52)],
   "venta_media_4w": [last_row["venta_media_4w"].values[0]],
   "venta_suma_4w": [last_row["venta_suma_4w"].values[0]]
})

pred = model.predict(next_input)[0]
prob = model.predict_proba(next_input)[0][1]

print(f"\n=== PREDICCIÓN ===")
print(f"¿Tendrá venta la próxima semana?: {'Sí' if pred else 'No'}")
print(f"Probabilidad de venta: {prob:.2%}")

# Mostrar importancia de variables
feature_importance = pd.DataFrame({
   'feature': features,
   'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n=== IMPORTANCIA DE VARIABLES ===")
print(feature_importance)

# Mostrar histórico reciente
print("\n=== HISTÓRICO RECIENTE ===")
print(data_model[['inicio_semana', 'venta_binaria', 'venta_semana']].tail())

# Interpretación automática
print("\n=== INTERPRETACIÓN ===")
if len(data_model) < 20:
   print("⚠️  ALERTA: Pocos datos históricos para entrenar el modelo")
if y.sum() < 3:
   print("⚠️  ALERTA: Muy pocas semanas con venta en el histórico")
if prob < 0.3:
   print("📉 Probabilidad BAJA de venta (modelo conservador)")
elif prob > 0.7:
   print("📈 Probabilidad ALTA de venta")
else:
   print("📊 Probabilidad MEDIA de venta (zona de incertidumbre)")

Registros totales: 28
Distribución de ventas: count    28.000000
mean      0.464286
std       0.838082
min       0.000000
25%       0.000000
50%       0.000000
75%       1.000000
max       3.000000
Name: venta_semana, dtype: float64
Proporción de semanas con venta: 28.57%

=== DIAGNÓSTICO INICIAL ===
Semanas con venta: 8
Semanas sin venta: 20
Registros para modelar: 24

=== VALIDACIÓN CRUZADA ===
Fold 1: Train=6, Test=6
  Train - Ventas: 3/6 (50.0%)
  Test - Ventas: 1/6 (16.7%)
  Precisión: 83.3%
Fold 2: Train=12, Test=6
  Train - Ventas: 4/12 (33.3%)
  Test - Ventas: 0/6 (0.0%)
  Precisión: 100.0%
Fold 3: Train=18, Test=6
  Train - Ventas: 4/18 (22.2%)
  Test - Ventas: 0/6 (0.0%)
  Precisión: 100.0%

Precisión promedio: 94.44% (+/- 15.71%)

=== PREDICCIÓN ===
¿Tendrá venta la próxima semana?: No
Probabilidad de venta: 16.67%

=== IMPORTANCIA DE VARIABLES ===
          feature  importance
0           lag_1           0
1           lag_2           0
2           lag_3           0
3       